In [1]:
import os
import nibabel as nib
import SimpleITK as sitk
import tqdm

# Print the working directory
cwd = os.getcwd()
print(cwd)

/home/alex/src


In [2]:
# get os.listdir excluding hidden files
def listdir_nohidden(path):
    list_dir = [f for f in os.listdir(path) if not f.startswith('.')]
    return list_dir

In [12]:
input_folder

'../data/datasets/ImaGenoma/T1_b'

In [13]:
# Define the root folder path
root_folder = '../data/datasets/ImaGenoma'

# Define the dataset name:
dataset = 'T1'

# Define the for loop to iterate through the folders
for folder in listdir_nohidden(root_folder):
    
    # Check if the item in the root folder is a directory and that it does not end with '_b':
    if os.path.isdir(os.path.join(root_folder, folder)) and not folder.endswith('_b'):
    
        # Get the path to the input folder
        input_folder = os.path.join(root_folder, folder)

        # Define the output folder for the corrected images
        output_folder = os.path.join(root_folder, folder + "_b")

        # Create the output folder if it does not exist
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        
        # Define the for loop to iterate through the Nifti images in the input folder
        for id in tqdm.tqdm(listdir_nohidden(input_folder)):
            
            ID_dir = os.path.join(input_folder, id)
            # make tqdm progress bar
            for filename in listdir_nohidden(ID_dir):
                
                # Check if the file is a Nifti image
                if filename.endswith('.nii') or filename.endswith('.nii.gz'):
                    # Get the path to the input image
                    input_image_path = os.path.join(ID_dir, filename)

                    # Load the input image using NiBabel
                    nifti_image = nib.load(input_image_path)

                    # Get the image data and the image affine matrix
                    image_data = nifti_image.get_fdata()
                    affine_matrix = nifti_image.affine

                    # Convert the image data to a SimpleITK image
                    sitk_image = sitk.GetImageFromArray(image_data)

                    # Perform N4 bias field correction on the input image using SimpleITK
                    corrected_image = sitk.N4BiasFieldCorrection(sitk_image)

                    # Convert the corrected image data to a Nifti image using NiBabel
                    corrected_image_data = sitk.GetArrayFromImage(corrected_image)
                    corrected_nifti_image = nib.Nifti1Image(corrected_image_data, affine_matrix)

                    # Make a folder for each id of the corrected images folder
                    corrected_id_folder = os.path.join(output_folder, id)
                    if not os.path.exists(corrected_id_folder):
                        os.makedirs(corrected_id_folder)
                    
                    # Get the path to the output image
                    output_image_path = os.path.join(corrected_id_folder, filename)

                    # Save the corrected Nifti image to the output folder using NiBabel
                    nib.save(corrected_nifti_image, output_image_path)  

  0%|          | 0/1022 [00:00<?, ?it/s]


OSError: Expected 7096320 bytes, got 3407520 bytes from ../data/datasets/ImaGenoma/T1/001065-00/3D_T1WI_MPRAGE.nii
 - could the file be damaged?